# Week 3: Divergence and vorticity
*MAQ - 32806, Chiel van Heerwaarden & Imme Benedict, 2017*

In this assignment you will investigate the concepts of divergence and vorticity. For this, you will use again the ECMWF ERA-Interim data from the 1st of January 2016 3:00. Note the new `get_pressure_index` function. This makes it easier to retrieve data at a certain pressure level.
___
First, we load the required packages.

In [ ]:
# Loading the packages.
import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import netCDF4 as nc            # NetCDF is the data format of the meteorological data that we use.
import matplotlib.pyplot as plt # Matplotlib is a scientific plotting package.
import datetime                 # Datetime is a package to deal with dates.
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.colors import LinearSegmentedColormap
from ipywidgets import interact
import warnings
warnings.filterwarnings('ignore')

# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%matplotlib inline
plt.rcParams.update({'font.size': 11})          # Set the standard font size of the plots to 11pt.
plt.rcParams.update({'figure.figsize': [13,6]}) # Set the standard figure size.

In [ ]:
# Create custom color map similar to the NCAR NCL WhiteBlueGreenYellowRed
cdict = {'red':   ((   0/253., 255./255., 255./255.),
                   (  36/253., 157./255., 157./255.),
                   (  72/253.,  72./255.,  72./255.),
                   ( 108/253.,  73./255.,  73./255.),
                   ( 145/253., 250./255., 250./255.),
                   ( 181/253., 245./255., 245./255.),
                   ( 217/253., 211./255., 211./255.),
                   ( 253/253., 146./255., 146./255.)),
         'green': ((   0/253., 255./255., 255./255.),
                   (  36/253., 218./255., 218./255.),
                   (  72/253., 142./255., 142./255.),
                   ( 108/253., 181./255., 181./255.),
                   ( 145/253., 232./255., 232./255.),
                   ( 181/253., 106./255., 106./255.),
                   ( 217/253.,  31./255.,  31./255.),
                   ( 253/253.,  21./255.,  21./255.)),
         'blue':  ((   0/253., 255./255., 255./255.),
                   (  36/253., 247./255., 247./255.),
                   (  72/253., 202./255., 202./255.),
                   ( 108/253.,  70./255.,  70./255.),
                   ( 145/253.,  92./255.,  92./255.),
                   ( 181/253.,  45./255.,  45./255.),
                   ( 217/253.,  40./255.,  40./255.),
                   ( 253/253.,  25./255.,  25./255.))}

my_cmap = LinearSegmentedColormap('my_colormap', cdict,256)

___
# Understanding divergence, vorticity and deformation

Before studying weather data, you will first study the two-dimensional flow patterns associated with divergence $\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y}$, vorticity $\dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y}$, and the two components of deformation $\dfrac{\partial u}{\partial x} - \dfrac{\partial v}{\partial y}$ and $\dfrac{\partial v}{\partial x} + \dfrac{\partial u}{\partial y}$. Every infinitisimal velocity field can be described as a linear combination of these four. See Holton section 1.5 for a full explanation.

In [ ]:
# Interactive widget for divergence, vorticity, and deformation.
def plot_streamlines(div=0, zeta=0, d1=0, d2=0):
    n = 65
    x = np.linspace(-1., 1., n)
    y = np.linspace(-1., 1., n)
    u = 0.5*(div +d1)*x[np.newaxis, :] + 0.5*(d2 -zeta)*y[:,np.newaxis]
    v = 0.5*(zeta+d2)*x[np.newaxis, :] + 0.5*(div-d1  )*y[:,np.newaxis]
    U = (u**2 + v**2)**.5

    fig = plt.figure(figsize=(8,6))
    ax = plt.subplot(111, aspect='equal')
    cb = ax.contourf(x, y, U, 21)
    fig.colorbar(cb)
    ax.streamplot(x, y, u, v, color='k', arrowsize=2.)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_xlim(-1.,1.)
    ax.set_ylim(-1.,1.)
    
interact(plot_streamlines,
         div=(-1,1,0.1), zeta=(-1,1,0.1), d1=(-1,1,0.1), d2=(-1,1,0.1));

___
# Setting up Python and loading the data
With the code below, you initialize the world map with the desired coordinates.

In [ ]:
# We define a map region on the northern hemisphere that we use through the assignment.
lon_start =  - 70.
lon_end   =    30.
lat_start =    20.
lat_end   =    71.

___
Now, you load the data and read out the desired variables from a NetCDF file from the ECMWF ERA-Interim data archive.

In [ ]:
# Loading the ERA data.
nc_file = nc.Dataset("era_data.nc", "r")
lat = nc_file.variables["latitude"][:]
lon = nc_file.variables["longitude"][:]
p = nc_file.variables["level"][:]*100.
t = 0
time = nc.num2date(nc_file.variables["time"][t], nc_file.variables["time"].units)
print("The time is: {}".format(time))

# We define the step for the quivering.
nqstep = 4

# With the code below, we roll the map, to get the 0 meridian,
# rather than the Pacific in the middle.
nroll = lon.size//2
lon = np.roll(lon, nroll)
lon = np.where(lon>=180., lon-360., lon)

# We load 3D fields of the two horizontal wind components, the geopotential and the temperature.
u     = np.roll(nc_file.variables["u"][t,:,:,:], nroll, -1)
v     = np.roll(nc_file.variables["v"][t,:,:,:], nroll, -1)
omega = np.roll(nc_file.variables["w"][t,:,:,:], nroll, -1)
Phi   = np.roll(nc_file.variables["z"][t,:,:,:], nroll, -1)
T     = np.roll(nc_file.variables["t"][t,:,:,:], nroll, -1)
slp   = np.roll(nc_file.variables["msl"][t,:,:], nroll, -1)
pr    = np.roll(nc_file.variables["lsp"][t,:,:], nroll, -1)\
      + np.roll(nc_file.variables["cp" ][t,:,:], nroll, -1)

nc_file.close()

def get_lat_index(lat_plot):
    return abs(lat-lat_plot).argmin()

def get_pressure_index(p_plot):
    return abs(p-p_plot).argmin()

nq = 3
qscale = 2000.

___
# Plotting the temperature, geopotential and wind vectors
As a starting point you find below the temperature and geopotential at 500 hPa.

In [ ]:
# Plot of T and Phi
n = get_pressure_index(50000.)
quiver = True # Set this flag to True if you want to enable the wind arrows.

my_projection = ccrs.PlateCarree(central_longitude=0)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, T[n,:,:], 15, cmap=my_cmap) # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.
if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'T and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout() # With this function we make the figure fit as good as possible.

___
# Assignment
In this assignment you are going to analyze the structure of vorticity and divergence surrounding the cyclone at 48N, 20W. Before doing so, you will get familiar with divergence, vorticity and deformation.

## A first glimpse on vorticity, divergence and deformation
Spatial variations in the velocity field can alter a moving air parcel in three possible ways. The variations can introduce *divergence*, *vorticity* and *deformation*. Use the applet at the top of the notebook and move the sliders to change the relative contribution of eacht of the three. The values in this applet are scaled and range between -1 and 1. The purpose of this exercise is to understand the patterns, not to understand its magnitudes. Make sure to read Holton section 1.5.

1. Explain the flow pattern for a flow with only divergence. What happens if you change the sign?
1. Explain the flow pattern for a flow with only vorticity. What happens if you change the sign?
1. Explain the flow pattern for a flow with only deformation. What happens if you change the sign?
1. Explain the flow pattern for a flow with divergence and vorticity. What happens if you change the relative magnitude of one of the two? Which combinations do you find in high and low pressure systems on the Northern hemisphere? Which not?
1. Combine vorticity and one of the two deformation components and put both at the same magnitude. What do you observe? How is rotation generated in such a flow pattern?

## Vorticity on the weather map
In this exercise, focus your answers on the surroundings of the cyclone at 48N and 20W.
1. Calculate the relative vorticity $\zeta \equiv \dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y}$, and plot it at 850, 500 and 300 hPa. Explain the patterns and the differences you observe.
2. Calculate and plot the planetary vorticity $f$ at the 500 hPa level. How does it look at 300 hPa?
3. Plot the absolute vorticity $\eta \equiv \zeta + f$ at 300 hPa. What do you observe? Which of its two components dominates?

## Divergence and vertical velocity on the weather map
In this exercise, focus your answers on the surroundings of the cyclone at 48N and 20W.
1. Calculate the horizontal divergence $\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y}$ and plot it at 850, 500 and 300 hPa. Explain the patterns and the differences you observe.
2. Plot the vertical velocity $\omega$ at the 500 hPa level. What do you observe? How does it compare to the divergence plots?
3. Plot the precipitation `pr` and $\omega$ at the 500 hPa level in the same plot and discuss whether and why the two look similar.
4. Make a vertical cross section along latitude of 48N of the temperature and plot the wind vectors in there with the `quiver` function. TIP: multiply $\omega$ with -10 to be able to observe changes in the vertical velocity. Does the vertical profile corroborate your earlier explanations?
___
Work out the assignment below.
____

In [ ]:
lonrad = np.deg2rad(lon)
latrad = np.deg2rad(lat)
cos_lat = np.cos(latrad)

a_earth = 6.37e6
dvdx = (a_earth*cos_lat[np.newaxis,:,np.newaxis])**(-1) \
     * np.gradient(v, axis=2) / np.gradient(lonrad[np.newaxis, np.newaxis, :], axis=2)
dudy = (a_earth)**(-1) \
     * np.gradient(u, axis=1) / np.gradient(latrad[np.newaxis, :, np.newaxis], axis=1)
zeta = dvdx - dudy

Omega = 7.292e-5
f = 2.*Omega*np.sin(latrad)
ff = f[:,np.newaxis]*np.ones((lat.shape[0], lon.shape[0]))

In [ ]:
# Plots exercise 2: Relative vorticity
quiver = False
zeta_range = np.linspace(-3e-4, 3e-4, 31)

# FIGURE 1
n = get_pressure_index(85000.)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, zeta[n,:,:], zeta_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

# FIGURE 2
n = get_pressure_index(50000.)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, zeta[n,:,:], zeta_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

# FIGURE 3
n = get_pressure_index(30000.)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, zeta[n,:,:], zeta_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

In [ ]:
# Exercise 2: Planetary and absolute vorticity.
n = get_pressure_index(30000.)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, ff, zeta_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$f$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, zeta[n,:,:] + ff, zeta_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$f + \zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

In [ ]:
dudx = (a_earth*cos_lat[np.newaxis,:,np.newaxis])**(-1) \
     * np.gradient(u, axis=2) / np.gradient(lonrad[np.newaxis, np.newaxis, :], axis=2)
dvdy = (a_earth)**(-1) \
     * np.gradient(v, axis=1) / np.gradient(latrad[np.newaxis, :, np.newaxis], axis=1)

divh = dudx + dvdy

In [ ]:
# Exercise 3.
quiver = False
div_range = np.linspace(-7e-5, 7e-5, 21)

# FIGURE 1
n = get_pressure_index(85000.)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, divh[n,:,:], div_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.) 
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$\nabla \cdot V$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

# FIGURE 2
n = get_pressure_index(50000.)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, divh[n,:,:], div_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$\nabla \cdot V$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

# FIGURE 3
n = get_pressure_index(30000.)

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, divh[n,:,:], div_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#cz = ax1.contour(lon, lat, slp, 15, colors='w', linewidths=1.5) # We plot the geopotential in contours.

ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'$\nabla \cdot V$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

In [ ]:
# Omega at 500 hPa with divergence
n = get_pressure_index(50000.)

omega_range = np.linspace(-1., 1., 21)

quiver = False

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, omega[n,:,:], omega_range, cmap=plt.cm.seismic, extend='both') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='k', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

cp = ax1.contour(lon, lat, slp, 15, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cp, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

ax1.set_title(r'$\omega$ and $\Phi$ at p = {0:.0f} hPa, and SLP'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

In [ ]:
# Omega at 500 hPa with precipitation
n = get_pressure_index(50000.)

pr_range = np.linspace(0, 6., 21)

quiver = False

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

ax1.set_xticks(np.arange(-180, 181, 20), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 20), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

cb = ax1.contourf(lon, lat, pr*1000, pr_range, cmap=plt.cm.Greens, extend='max') # We plot a colormesh using the gist_ncar colormap.
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

if quiver:
    lons, lats = np.meshgrid(lon, lat)
    qu = ax1.quiver(lons[::nq, ::nq], lats[::nq, ::nq], u[n,::nq,::nq], v[n,::nq,::nq],\
                    pivot='mid', width=1.2e-3, scale=1000.)
cz = ax1.contour(lon, lat, Phi[n,:,:], 15, colors='k', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

cp = ax1.contour(lon, lat, slp, 15, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cp, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

ax1.set_title(r'$\omega$ and $\Phi$ at p = {0:.0f} hPa, and SLP'.format(p[n]/100.)); # We add a title to the plot.
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(lat_start, lat_end)
fig1.tight_layout()

In [ ]:
# Vertical cross section temperature and wind.
nqv = 3
lat_plot = 48.
j_lat = abs(lat - lat_plot).argmin()

lonp, pp = np.meshgrid(lon, p)

fig1 = plt.figure()
ax1 = plt.subplot(111)
cT = ax1.contourf(lon, p, T[:,j_lat,:], 21)
fig1.colorbar(cT)
ax1.quiver(lonp[:,::nqv], pp[:,::nqv], u[:,j_lat,::nqv], -10*omega[:,j_lat,::nqv], 
          pivot='mid', width=1.2e-3, scale=1000)
ax1.invert_yaxis()
ax1.set_xlim(lon_start, lon_end)
ax1.set_ylim(100000, 1000);
ax1.set_xlabel('lon')
ax1.set_ylabel('p')
fig1.tight_layout()